In [2]:
import json
import os
import numpy as np
import pandas as pd

In [93]:
train = pd.read_json('train.json', typ='frame', encoding='utf-8')
tag_song_map = pd.read_json('tag_song_pair.json', typ='frame', orient='table', encoding='utf-8')

tags = tag_song_map['tag']
tag_plylst_map =  train[['id', 'tags']]

In [94]:
tag_plylst_map

,id,tags
0,61281,[락]
1,10532,"[추억, 회상]"
2,76951,"[까페, 잔잔한]"
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,..."
4,27616,[댄스]
...,...,...
115066,120325,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]"
115067,106976,[일렉]
115068,11343,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]"
115069,131982,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]"


In [95]:
# unnest tags
tag_plylst_map_unnest = np.dstack(
    (
        np.repeat(tag_plylst_map.id.values, list(map(len, tag_plylst_map.tags))), 
        np.concatenate(tag_plylst_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
tag_plylst_map = pd.DataFrame(data = tag_plylst_map_unnest[0], columns = tag_plylst_map.columns)
tag_plylst_map['id'] = tag_plylst_map['id'].astype(str)

# unnest 객체 제거
del tag_plylst_map_unnest

In [126]:
tag_plylst_map

,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


In [103]:
tag_cnt = tag_plylst_map.groupby('tags').tags.count().reset_index(name = 'mapping_cnt')
tag_cnt['tags'] = tag_cnt['tags'].astype(str)
taglist = np.array(list(tag_cnt['tags']))

tag_plylst_group = pd.DataFrame(data=taglist, columns=['tag'])
tag_plylst_group['plylst_id']=np.nan

In [104]:
print(tag_plylst_group)

          tag  plylst_id
0                    NaN
1          00        NaN
2         007        NaN
3      007시리즈        NaN
4         00s        NaN
...       ...        ...
29155      힙해        NaN
29156     힙힙힙        NaN
29157    힛뎀포크        NaN
29158     힛뎀폭        NaN
29159    힛뎀폭스        NaN

[29160 rows x 2 columns]


In [105]:
tmp = tag_plylst_group.to_numpy()
for i in range(len(taglist)):
    tmp[i,1]=set()
print(tmp)
print(len(tmp))

[['' set()]
 ['00' set()]
 ['007' set()]
 ...
 ['힛뎀포크' set()]
 ['힛뎀폭' set()]
 ['힛뎀폭스' set()]]
29160


In [106]:
tag_plylst_id = train[['tags', 'id']]

In [111]:
for i in range(len(tag_plylst_id)):
    for tag in tag_plylst_id.loc[i]['tags']:
        tag_index = np.where(taglist==tag)
        tmp[tag_index[0][0]][1].add(tag_plylst_id.loc[i]['id'])
    if(i%1000==0): print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000


In [119]:
tag_plylst_pair = pd.DataFrame(data=tmp, columns=['tag','plylst_id'])
tag_plylst_pair.to_json('tag_plylst_pair.json', orient='table')

In [120]:
tag_plylst_pair

,tag,plylst_id
0,,{95032}
1,00,"{123921, 38363, 3324, 81830}"
2,007,"{46937, 89985, 45506}"
3,007시리즈,"{12845, 139999}"
4,00s,"{45561, 61686, 46831}"
...,...,...
29155,힙해,"{119138, 71426, 76132, 40247, 152409}"
29156,힙힙힙,"{59850, 45778}"
29157,힛뎀포크,{37691}
29158,힛뎀폭,{37691}
